In [1]:
%cd -q ..

import numpy as np
import pandas as pd

from model.helpers import load_params
from model.model import Model
from model.inpatients import InpatientsModel
from model.outpatients import OutpatientsModel
from model.aae import AaEModel
from model.health_status_adjustment import HealthStatusAdjustmentInterpolated

pd.set_option("display.precision", 1)

In [2]:
import logging

logging.basicConfig(
    format="%(asctime)s %(levelname)-8s %(message)s",
    level=logging.INFO,
    datefmt="%Y-%m-%d %H:%M:%S",
)

In [3]:
params = load_params("queue/sample_params.json")
run_params = Model.generate_run_params(params)
hsa = HealthStatusAdjustmentInterpolated(
    f"data/2019/{params['dataset']}", params["start_year"]
)

# Helper Methods

Below are some helper methods for turning the results into useful Pandas `DataFrames`.

In [4]:
def run_model(model, i=0):
    return {
        k0: pd.DataFrame([{**dict(k1), "value": v1} for k1, v1 in v0.items()])
        for k0, v0 in model(params, "data", hsa, run_params).go(i).items()
    }

In [5]:
def default_pod_measure_pivot(agg):
    default_results = (
        agg["default"]
        .groupby(["pod", "measure"], as_index=False)
        .agg({"value": sum})
        .pivot(index=["pod"], columns="measure")
        .fillna(0)
    )
    default_results.loc["total"] = default_results.sum()

    return default_results

In [6]:
def step_counts_simple(agg):
    step_counts = (
        agg["step_counts"]
        .drop(columns=["strategy", "activity_type"])
        .groupby(["change_factor", "measure"], as_index=False)
        .sum()
        .pivot(index="change_factor", columns="measure")
        .loc[agg["step_counts"]["change_factor"].unique()]
    )
    step_counts.loc["total"] = step_counts.sum()

    return step_counts

# Running the models

## Inpatients

Running the principal model run (model run 0) for the Inpatients data

In [7]:
ip_agg = run_model(InpatientsModel)

### Aggregate Results

The available aggregations for inpatients are:

In [8]:
list(ip_agg.keys())

['default',
 'sex+age_group',
 'sex+tretspef',
 'bed_occupancy',
 'theatres_available',
 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [9]:
default_pod_measure_pivot(ip_agg)

value                                 
measure                   admissions attendances   beddays procedures
pod                                                                  
ip_elective_admission         6698.0         0.0   27248.0     3677.0
ip_elective_daycase          54796.0         0.0   54796.0    29092.0
ip_maternity_admission        6740.0         0.0   31978.0     3566.0
ip_non-elective_admission    45255.0         0.0  162992.0     6358.0
op_procedure                     0.0       923.0       0.0        0.0
total                       113489.0       923.0  277014.0    42693.0

### Estimate of effects

In [10]:
step_counts_simple(ip_agg)

value          
measure                    admissions   beddays
change_factor                                  
baseline                      95786.0  305599.0
demographic_adjustment        17940.4   56818.8
health_status_adjustment         85.7     316.2
covid_adjustment               1584.6    4101.2
expat                          -209.8    -706.4
repat                          2321.9    7531.1
waiting_list_adjustment         166.4     227.2
baseline_adjustment               0.0       0.0
non-demographic_adjustment    11858.2   38431.3
activity_avoidance           -15121.5  -67005.4
efficiencies                   -923.0  -68299.0
total                        113489.0  277014.0

## Outpatients

Running the principal model run (model run 0) for the Outpatients data

In [11]:
op_agg = run_model(OutpatientsModel)

### Aggregate Results

The available aggregations for outpatients are:

In [12]:
list(op_agg.keys())

['default', 'sex+age_group', 'sex+tretspef', 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [13]:
default_pod_measure_pivot(op_agg)

value                 
measure      attendances tele_attendances
pod                                      
op_first          157283            20166
op_follow-up      308500            41876
op_procedure       44988                0
total             510771            62042

### Estimate of effects

In [14]:
step_counts_simple(op_agg)

value                 
measure                  attendances tele_attendances
change_factor                                        
baseline                    451052.0          19802.0
demographic_adjustment       70229.8           3370.2
health_status_adjustment       222.4             10.7
covid_adjustment             55326.6           2370.5
expat                        -1531.7            -68.0
repat                        11444.3            501.8
waiting_list_adjustment       7750.6            341.3
baseline_adjustment              0.0              0.0
activity_avoidance          -46002.9          -2006.4
efficiencies                -37720.0          37720.0
total                       510771.0          62042.0

## Accident and Emergency

Running the principal model run (model run 0) for the A&E data

In [15]:
ae_agg = run_model(AaEModel)

### Aggregate Results

The available aggregations for outpatients are:

In [16]:
list(ae_agg.keys())

['default', 'sex+age_group', 'step_counts']

We can view the default aggregations, summarising to pod/measure

In [17]:
default_pod_measure_pivot(ae_agg)

value        
measure     ambulance walk-in
pod                          
aae_type-01     36790   71744
aae_type-02         1    1243
aae_type-03       206   18801
aae_type-04         2    1021
total           36999   92809

### Estimate of effects

In [18]:
step_counts_simple(ae_agg)

,value
measure,arrivals
change_factor,
baseline,122556.0
demographic_adjustment,13555.9
health_status_adjustment,33.6
covid_adjustment,3672.8
expat,-2799.1
repat,2743.1
baseline_adjustment,0.0
activity_avoidance,-9954.3
